In [1]:
import requests
import json
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
import pandas as pd
from bokeh.models import HoverTool
import ipywidgets
from IPython.core.display import display, clear_output
output_notebook()

Loading BokehJS ...

In [2]:
align_kw = dict(
    _css = (('.widget-label', 'min-width', '700px'),),
    margin = '0px 0px 5px 12px',
    width='500px'
)

pred_point = ipywidgets.IntSlider(description='Number of Prediction Point(s):', value=5, min=1, max=10, **align_kw)
opr = ipywidgets.FloatSlider(description='Overnight Policy Rate (OPR):', value=3.17, min=0.0, max=10.0, **align_kw)
cpi = ipywidgets.FloatSlider(description='Consumer Price Index (CPI):', value=115, min=0.0, max=200.0, **align_kw)
usd = ipywidgets.FloatSlider(description='USD Rate:', value=4.2, min=0.0, max=10.0, **align_kw)
klci = ipywidgets.FloatSlider(description='KLCI Index:', value=1800, min=0.0, max=3000.0, **align_kw)
grant = ipywidgets.FloatSlider(description='Grant Fund:', value=50000000, min=0.0, max=500000000.0, **align_kw)

btn = ipywidgets.Button(description='Simulate')

container = ipywidgets.VBox(children=[pred_point,opr,cpi,usd,klci,grant,btn],display='flex')
display(container)

def btn_click(sender):
    api_key = "Vk68iUzss/aDOtFinY763pSPBbPhs0DE9+bhUQjDFAWiu+sIchb2E7KBjNp7do6RmU+Nl0kH1NsqmCUJjoUlJw=="
    headers = {'Content-Type':'application/json', 'Authorization':'Bearer '+api_key}
    url = 'https://ussouthcentral.services.azureml.net/workspaces/510753582ca648c5952ae3538bef64de/services/e1ca94bc7b32408d9bd54be02a255aee/execute?api-version=2.0&details=true'
    jinput = {
      "Inputs": {
        "input1": {
          "ColumnNames": [
            "numberOfPoints",
            "overnightInterest",
            "cpi",
            "usd",
            "klciIndex",
            "fund"
          ],
          "Values": [
            [
              pred_point.value,
              opr.value,
              cpi.value,
              usd.value,
              klci.value,
              grant.value
            ]
          ]
        }
      },
      "GlobalParameters": {}
    }
    r = requests.post(url, data=json.dumps(jinput), headers=headers)
    global data
    data = r.json()
    colname = data['Results']['output1']['value']['ColumnNames']
    values = data['Results']['output1']['value']['Values']
    datapoints = { c:[i[idx] for i in values] for idx,c in enumerate(colname) }
    datapoints['date'] = list(map(pd.to_datetime,datapoints['date']))
    datapoints['GDP'] = list(map(lambda x: float(x)/1000,datapoints['GDP']))
    datapoints['confidentIntervalLower'] = list(map(lambda x: float(x)/1000 if x else None,datapoints['confidentIntervalLower']))
    datapoints['confidentIntervalUpper'] = list(map(lambda x: float(x)/1000 if x else None,datapoints['confidentIntervalUpper']))

    source_gdp = ColumnDataSource(
            data=dict(
                quarter=[x.strftime("%Y-%m") for x in datapoints['date']],
                GDP=["{0:.2f}mil".format(x) for x in datapoints['GDP']],
                x=datapoints['date'],    
                y=datapoints['GDP'])
            )

    patch_quarter=datapoints['date'][25:]+datapoints['date'][25:][::-1]
    patch_gdp = [datapoints['GDP'][25]]+datapoints['confidentIntervalLower'][26:]+datapoints['confidentIntervalUpper'][26:][::-1]+[datapoints['GDP'][25]]
    source_gdp_interval = ColumnDataSource(
            data=dict(
                quarter=[x.strftime("%Y-%m") for x in patch_quarter],
                GDP=["{0:.2f}mil".format(x) for x in patch_gdp],
                x=patch_quarter,    
                y=patch_gdp)
            )

    hover = HoverTool(
            tooltips=[
                ("(quarter, GDP)", "(@quarter, @GDP)"),
            ]
        )

    p = figure(title="GDP Forecast", x_axis_type="datetime", width=800, height=500)
    p.add_tools(hover)
    p.line(x='x',y='y',source=source_gdp)
    p.yaxis.axis_label = 'GDP (mil USD)'
    p.patch(x='x', 
            y='y',
            alpha=0.2,
            color='grey',
            line_width=2,line_dash='dashed',
           source=source_gdp_interval
           )
    clear_output()
    show(p)
    

btn.on_click(btn_click)